In [2]:
import sys
sys.path.insert(0, '..') # 상위 폴더를 python import path에 추가

In [3]:
import inspect
from typing import TypeVar


# https://stackoverflow.com/questions/13520421/recursive-dotdict
class attrdict(dict):
    """
    a dictionary that supports dot operation
    as well as dictionary access operation
    usage: d = attrdict() or d = attrdict({'val1':'first'})
    set attributes: d.val2 = 'second' or d['val2'] = 'second'
    get attributes: d.val2 or d['val2']
    """

    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


def attr(obj) -> tuple[attrdict, attrdict, attrdict, attrdict]:
    """Tool for python object inspection. Python object has its namespace and attribute.
    Python's built-in function 'dir' is useful, but inconvenient for object namespace analysis.
    So, this function renovates python's built-in functions like dir, var, ...

    Returns:
        tuple: (obj's state_types, obj's callable_signatures, state_values, and bounded callables)
    """
    all_attr = {}
    for attribute in dir(obj):
        if not attribute.startswith("_"):
            try:
                all_attr[attribute] = getattr(obj, attribute)
            except AttributeError:
                continue

    methods = dict([(k, v) for k, v in all_attr.items() if callable(v)])

    signatures = {}
    for k, v in all_attr.items():
        if callable(v):
            try:
                signatures[k] = inspect.signature(v)  # may occur ValueError
            except ValueError:
                signatures[k] = "No signature available for built-in method"

    state_keys = sorted(list(set(all_attr.keys()) - set(methods.keys())))
    state_types = dict([(k, type(getattr(obj, k))) for k in state_keys])
    state_values = dict([(k, getattr(obj, k)) for k in state_keys])

    return attrdict(state_types), attrdict(signatures), attrdict(state_values), attrdict(methods)

# Models
https://developers.google.com/mediapipe/solutions/vision/hand_landmarker#models   
   
#####  HandLandmarker  
![HandLandmarker](https://developers.google.com/static/mediapipe/images/solutions/hand-landmarks.png)

##### 활용 사례
https://github.com/GasbaouiMohammedAlAmin/Finger-Counter-using-Hand-Tracking-And-Open-cv/blob/main/README.md   
   
![Screenshop](https://github.com/GasbaouiMohammedAlAmin/Finger-Counter-using-Hand-Tracking-And-Open-cv/raw/main/fingerCounting.png)

### [HandLandmarkerResult 분석](https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python#handle_and_display_results)
 - mediapipe.tasks.python.vision.hand_landmarker.HandLandmarkerResult
 - public abstract class HandLandmarkerResult   
 - Represents the hand landmarks deection results generated by HandLandmarker.


##### main.py

In [4]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.tasks.python.vision.hand_landmarker import HandLandmarkerResult, HandLandmarker
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import logging, coloredlogs

from settings import *  # loading settings.py data to current 'main' namespace



2023-11-30 04:13:13 AERO15 settings[31488] INFO importing settings.py
2023-11-30 04:13:13 AERO15 settings[31488] INFO root 폴더: c:\Users\kclee\OneDrive\Develope\2023-2\OpenSW\OSS-project-13\notebook\..
2023-11-30 04:13:13 AERO15 settings[31488] INFO finger_tips index: [4, 8, 12, 16, 20]


In [5]:
logger = logging.getLogger(__name__)
coloredlogs.install(level="DEBUG", logger=logger)  # logger 설정, logger.debug() 함수로 로그메시지 표시


# https://github.com/googlesamples/mediapipe/blob/main/examples/hand_landmarker/python/hand_landmarker.ipynb
def draw_fingercount_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness
    annotated_image = np.copy(rgb_image)

    # Loop through the detected hands to visualize.
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        handedness = handedness_list[idx]

        # Draw the hand landmarks.
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend(
            [landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks]
        )
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            hand_landmarks_proto,
            solutions.hands.HAND_CONNECTIONS,
            solutions.drawing_styles.get_default_hand_landmarks_style(),
            solutions.drawing_styles.get_default_hand_connections_style(),
        )

        # Get the top left corner of the detected hand's bounding box.
        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks]
        y_coordinates = [landmark.y for landmark in hand_landmarks]
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        # Draw handedness (left or right hand) on the image.
        cv2.putText(
            annotated_image,
            text=f"{handedness[0].category_name}",
            org=(text_x, text_y),
            fontFace=cv2.FONT_HERSHEY_DUPLEX,
            fontScale=FONT_SIZE,
            color=HANDEDNESS_TEXT_COLOR,
            thickness=FONT_THICKNESS,
            lineType=cv2.LINE_AA,
        )

    return annotated_image



base_options = python.BaseOptions(model_asset_path=root / "models/hand_landmarker.task")
options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=2)
detector: HandLandmarker = vision.HandLandmarker.create_from_options(options)  # hand detector 객체 생성

cap = cv2.VideoCapture(0)

while True:
    success, image_origin = cap.read()
    if success:
        image = cv2.cvtColor(image_origin, cv2.COLOR_BGR2RGB)
        rgb_frame = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
        detection_result: HandLandmarkerResult = detector.detect(rgb_frame)  # hand landmarker result (num_hands=2, 양손 가능)

        if detection_result:  # detection_result 로그데이터 생성
            logger.debug(f"detection_result:\n {detection_result}")

        annotated_image = draw_fingercount_on_image(  # 원본 카메라 영상에 detection 결과를 합성하는 듯~
            rgb_image=image_origin,
            detection_result=detection_result,
        )

        cv2.imshow(f"Finger Counter", annotated_image)  # 동영상 재생
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        print(f"cap.read() error")
        break

cap.release()
cv2.destroyAllWindows()


2023-11-30 04:13:17 AERO15 __main__[31488] DEBUG detection_result:
 HandLandmarkerResult(handedness=[[Category(index=1, score=0.9562605619430542, display_name='Left', category_name='Left')]], hand_landmarks=[[NormalizedLandmark(x=0.5808755159378052, y=0.6289367079734802, z=-5.679526680069102e-07, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5309454798698425, y=0.6397871971130371, z=0.017679041251540184, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5039349794387817, y=0.6143357753753662, z=0.025267453864216805, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.48012492060661316, y=0.600614070892334, z=0.025102363899350166, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.4580135643482208, y=0.5952573418617249, z=0.023774253204464912, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5354567170143127, y=0.5147090554237366, z=0.04379214346408844, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5008310675621033, y=0.4935920536518097, z=0.045884616672992

In [6]:
type(detection_result)

mediapipe.tasks.python.vision.hand_landmarker.HandLandmarkerResult

📌 HandLandmarkerResult는 mediapipe.tasks.python.vision.hand_landmarker.py 파일에서 데이터클래스로 정의됨

In [7]:
from mediapipe.tasks.python.vision.hand_landmarker import HandLandmarkerResult
from mediapipe.tasks.python.components.containers.category import Category
from mediapipe.tasks.python.components.containers.landmark import NormalizedLandmark, Landmark

#### 🚩 HandLandmarkerResult 클래스
```python
@dataclasses.dataclass
class HandLandmarkerResult:
  """The hand landmarks result from HandLandmarker, where each vector element represents a single hand detected in the image.

  Attributes:
    handedness: Classification of handedness.
    hand_landmarks: Detected hand landmarks in normalized image coordinates.
    hand_world_landmarks: Detected hand landmarks in world coordinates.
  """

  handedness: List[List[category_module.Category]]
  hand_landmarks: List[List[landmark_module.NormalizedLandmark]]
  hand_world_landmarks: List[List[landmark_module.Landmark]]
```

In [8]:
try:
    detection_result = HandLandmarkerResult()
except TypeError as e:
    logger.warning(e)


2023-11-30 04:13:43 AERO15 __main__[31488] WARNING HandLandmarkerResult.__init__() missing 3 required positional arguments: 'handedness', 'hand_landmarks', and 'hand_world_landmarks'


In [9]:
attr(Category)[0]

{'category_name': NoneType,
 'display_name': NoneType,
 'index': NoneType,
 'score': NoneType}

In [10]:
attr(NormalizedLandmark)[0]

{'presence': NoneType,
 'visibility': NoneType,
 'x': NoneType,
 'y': NoneType,
 'z': NoneType}

In [11]:
attr(Landmark())[0]

{'presence': NoneType,
 'visibility': NoneType,
 'x': NoneType,
 'y': NoneType,
 'z': NoneType}

#### 🚩 HandLandmarkerResult Example

In [12]:
HandLandmarkerResult(
    handedness=[[Category(index=1, score=0.9865500926971436, display_name="Left", category_name="Left")]],
    hand_landmarks=[
        [
            NormalizedLandmark(x=0.7697514295578003, y=0.6468967795372009, z=2.972396373479569e-07, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.668741762638092, y=0.6643492579460144, z=-0.040008749812841415, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5648773312568665, y=0.6172918677330017, z=-0.06133072078227997, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.4947565197944641, y=0.5577317476272583, z=-0.08032052218914032, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.4326040744781494, y=0.526371419429779, z=-0.0983131155371666, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5639973878860474, y=0.42474114894866943, z=-0.027148446068167686, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5079708099365234, y=0.32789477705955505, z=-0.05173785239458084, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.480471134185791, y=0.2685859501361847, z=-0.07474029809236526, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.4576159119606018, y=0.21509180963039398, z=-0.09316738694906235, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.6137695908546448, y=0.3705744445323944, z=-0.030617333948612213, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5645943880081177, y=0.23626628518104553, z=-0.049807462841272354, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5348491668701172, y=0.1555137187242508, z=-0.06909414380788803, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5079306960105896, y=0.08884336054325104, z=-0.08486079424619675, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.6730667352676392, y=0.34885501861572266, z=-0.040538471192121506, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.6328999996185303, y=0.22224289178848267, z=-0.06340891867876053, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.6061319708824158, y=0.14483919739723206, z=-0.08335975557565689, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5819135904312134, y=0.07867544889450073, z=-0.09826647490262985, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.741492509841919, y=0.349530965089798, z=-0.05467082932591438, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.7340850830078125, y=0.24912577867507935, z=-0.07368527352809906, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.7289763689041138, y=0.18323004245758057, z=-0.08507121354341507, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.7216171026229858, y=0.12328104674816132, z=-0.09445909410715103, visibility=0.0, presence=0.0),
        ]
    ],
    hand_world_landmarks=[
        [
            Landmark(x=0.046565815806388855, y=0.07342524081468582, z=0.006881324574351311, visibility=0.0, presence=0.0),
            Landmark(x=0.010594870895147324, y=0.07021936029195786, z=-0.007981224916875362, visibility=0.0, presence=0.0),
            Landmark(x=-0.014852593652904034, y=0.06191084906458855, z=-0.0166409183293581, visibility=0.0, presence=0.0),
            Landmark(x=-0.045510780066251755, y=0.04683224856853485, z=-0.025354573503136635, visibility=0.0, presence=0.0),
            Landmark(x=-0.06854020804166794, y=0.03167642652988434, z=-0.0283240657299757, visibility=0.0, presence=0.0),
            Landmark(x=-0.028759997338056564, y=0.014417590573430061, z=0.0012775243958458304, visibility=0.0, presence=0.0),
            Landmark(x=-0.04029626026749611, y=-0.010929192416369915, z=-0.008486682549118996, visibility=0.0, presence=0.0),
            Landmark(x=-0.050002872943878174, y=-0.025899965316057205, z=-0.02284790761768818, visibility=0.0, presence=0.0),
            Landmark(x=-0.059284575283527374, y=-0.0319778136909008, z=-0.05590397119522095, visibility=0.0, presence=0.0),
            Landmark(x=-0.007565224077552557, y=-0.0017618824495002627, z=0.008078535087406635, visibility=0.0, presence=0.0),
            Landmark(x=-0.021680276840925217, y=-0.038745105266571045, z=-0.006962615065276623, visibility=0.0, presence=0.0),
            Landmark(x=-0.03542018681764603, y=-0.056135863065719604, z=-0.02756108157336712, visibility=0.0, presence=0.0),
            Landmark(x=-0.047300662845373154, y=-0.06984193623065948, z=-0.05037245526909828, visibility=0.0, presence=0.0),
            Landmark(x=0.01660585217177868, y=-0.0115079116076231, z=-0.0009205307578667998, visibility=0.0, presence=0.0),
            Landmark(x=0.003100059926509857, y=-0.03883817046880722, z=-0.014047347940504551, visibility=0.0, presence=0.0),
            Landmark(x=-0.008930228650569916, y=-0.05718749761581421, z=-0.033243678510189056, visibility=0.0, presence=0.0),
            Landmark(x=-0.0204799622297287, y=-0.06876767426729202, z=-0.05420058220624924, visibility=0.0, presence=0.0),
            Landmark(x=0.036402590572834015, y=-0.003861718811094761, z=-0.007347520440816879, visibility=0.0, presence=0.0),
            Landmark(x=0.03788445517420769, y=-0.029805485159158707, z=-0.01170967984944582, visibility=0.0, presence=0.0),
            Landmark(x=0.03136403486132622, y=-0.04963252693414688, z=-0.02345612645149231, visibility=0.0, presence=0.0),
            Landmark(x=0.027734559029340744, y=-0.06272327154874802, z=-0.03760094568133354, visibility=0.0, presence=0.0),
        ]
    ],
);


### 참고 자료
[Hand landmarks detection guide for Python (https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python)](https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python)
- ~/anaconda3/envs/mediapipe/Lib/site-packages/mediapipe/tasks/python/vision/hand_landmarker.py
- ~/anaconda3/envs/mediapipe/Lib/site-packages/mediapipe/tasks/python/components/containers 폴더
